# Converter for the Financial Filings at the Sec

In [18]:
from zipfile import ZipFile
import pandas as pd

with ZipFile('importFiles/2020q2.zip') as myzip:
    with myzip.open('num.txt') as myfile1:
        dfNum = pd.read_table(myfile1,delimiter="\t")
    with myzip.open('pre.txt') as myfile2:
        dfPre = pd.read_table(myfile2,delimiter="\t")
    with myzip.open('sub.txt') as myfile3:
        dfSub = pd.read_table(myfile3,delimiter="\t", low_memory=False)
    with myzip.open('tag.txt') as myfile4:
        dfTag = pd.read_table(myfile4,delimiter="\t")

dfSym = pd.read_table('importFiles/ticker.txt', delimiter="\t", header=None, names=['symbol','cik'])        
dirname = "2020q2"

print(dfNum.size)
print(dfPre.size)
print(dfSub.size)
print(dfTag.size)
print(dfSym.size)

19779588
10966040
590796
841851
24168


In [2]:
from tabulate import tabulate

print(tabulate(dfNum.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+-------------------+-----------+---------+----------+--------+-------+------------------+------------+
|    | adsh                 | tag               | version   |   coreg |    ddate |   qtrs | uom   |            value |   footnote |
|----+----------------------+-------------------+-----------+---------+----------+--------+-------+------------------+------------|
|  0 | 0001308411-17-000026 | EntityPublicFloat | dei/2014  |     nan | 20170930 |      0 | USD   |      2.63776e+07 |        nan |
|  1 | 0001308411-17-000032 | EntityPublicFloat | dei/2014  |     nan | 20170930 |      0 | USD   |      2.63776e+07 |        nan |
|  2 | 0001308411-17-000030 | EntityPublicFloat | dei/2014  |     nan | 20170930 |      0 | USD   |      2.63776e+07 |        nan |
|  3 | 0001640334-17-002082 | EntityPublicFloat | dei/2014  |     nan | 20161231 |      0 | USD   |      2.40589e+06 |        nan |
|  4 | 0001059272-17-000037 | EntityPublicFloat | dei/2014  |     nan | 2016

In [3]:
print(tabulate(dfPre.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+----------+--------+--------+---------+---------+--------------+-----------+-----------------+------------+
|    | adsh                 |   report |   line | stmt   |   inpth | rfile   | tag          | version   | plabel          |   negating |
|----+----------------------+----------+--------+--------+---------+---------+--------------+-----------+-----------------+------------|
|  0 | 0000004904-17-000060 |        1 |      3 | CP     |       0 | H       | EntityDomain | dei/2014  | Entity [Domain] |          0 |
|  1 | 0000004904-17-000060 |        2 |      3 | IS     |       0 | H       | EntityDomain | dei/2014  | Entity [Domain] |          0 |
|  2 | 0000004904-17-000060 |        3 |      3 | CI     |       0 | H       | EntityDomain | dei/2014  | Entity [Domain] |          0 |
|  3 | 0000004904-17-000060 |        4 |      3 | CI     |       1 | H       | EntityDomain | dei/2014  | Entity [Domain] |          0 |
|  4 | 0000004904-17-000060 |        5 | 

In [4]:
print(tabulate(dfSub.head(100), headers='keys', tablefmt='psql'))

+----+----------------------+---------+-----------------------------------------------+-------+-------------+----------+------------------------+------------+-----------------------------------------+---------------------------------------+------------------+-------------+----------+------------------------+------------+-----------------------------------------+---------------------------------------+--------------+-----------+-------------+-------------------------------------------------+---------------+-------+--------+-------+--------+----------+------+------+----------+-----------------------+-----------+----------+---------------------+---------+-------------------------------------------------------------------------------------------------------------------------+
|    | adsh                 |     cik | name                                          |   sic | countryba   | stprba   | cityba                 | zipba      | bas1                                    | bas2            

In [5]:
print(tabulate(dfTag.head(100), headers='keys', tablefmt='psql'))

+----+--------------------------------------------------------------+--------------+----------+------------+------------------+--------+--------+-------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    | tag                                                          | vers

In [7]:
print(tabulate(dfSym.head(10), headers='keys', tablefmt='psql'))

+----+----------+---------+
|    | symbol   |     cik |
|----+----------+---------|
|  0 | aapl     |  320193 |
|  1 | msft     |  789019 |
|  2 | brk-b    | 1067983 |
|  3 | unh      |  731766 |
|  4 | jnj      |  200406 |
|  5 | v        | 1403161 |
|  6 | tsm      | 1046179 |
|  7 | xom      |   34088 |
|  8 | wmt      |  104169 |
|  9 | spy      |  884394 |
+----+----------+---------+


In [8]:
print(dfNum.iloc[0])
print(dfNum.columns.values)

adsh        0001308411-17-000026
tag            EntityPublicFloat
version                 dei/2014
coreg                        NaN
ddate                   20170930
qtrs                           0
uom                          USD
value                 26377573.0
footnote                     NaN
Name: 0, dtype: object
['adsh' 'tag' 'version' 'coreg' 'ddate' 'qtrs' 'uom' 'value' 'footnote']


In [9]:
from datetime import date, datetime, timedelta
from marshmallow import Schema, fields

class FinancialElementImportDto:
    label = ""          #Tag.doc
    concept = ""        #Num.tag
    info = ""           #Pre.plabel
    unit = ""           #Num.uom
    value = 0.0         #Num.value

class FinancialsDataDto:
    bs = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    cf = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    ic = []  #FinancialElementImportDto[] mapping key from Pre.stmt
    

class SymbolFinancialsDto:
    startDate = date.today()   #Sub.period 
    endDate = date.today()     #Sub.period + Sub.fp
    year = 0                   #Sub.fy
    quarter = ""               #Sub.fp
    symbol = ""                #Sub.cik -> Sym.cik -> Sym.symbol
    name = ""                  #Sub.name
    country = ""               #Sub.countryma
    city = ""                  #Sub.cityma
    data = FinancialsDataDto()
    
class FinancialElementImportSchema(Schema):
    label = fields.String()
    concept = fields.String()
    info = fields.String()
    unit = fields.String()
    value = fields.Int()
    
class FinancialsDataSchema(Schema):
    bs = fields.List(fields.Nested(FinancialElementImportSchema()))
    cf = fields.List(fields.Nested(FinancialElementImportSchema()))
    ic = fields.List(fields.Nested(FinancialElementImportSchema()))
    
class SymbolFinancialsSchema(Schema):
    startDate = fields.DateTime()
    endDate = fields.DateTime()
    year = fields.Int()
    quarter = fields.String()
    symbol = fields.String()
    name = fields.String()
    country = fields.String()
    city = fields.String()
    data = fields.Nested(FinancialsDataSchema)

In [10]:
import os

#isExist = os.path.exists("exportFiles/"+dirname)
isExist = 1 == 2
if isExist:
    for filename in os.listdir("exportFiles/"+dirname):
        with open(os.path.join("exportFiles/"+dirname, filename), 'r') as file:
            jsonStr = file.read()
            file.close()
        jsonStr = jsonStr.replace("'", '"').replace(", ",",").replace('\\r', '').replace('\\n', ' ')
        jsonStr = ''.join(jsonStr.splitlines())
        with open(os.path.join("exportFiles/"+dirname, filename), 'w') as file:    
            file.write(jsonStr)
            file.close()
    

# Create a single SymbolFinancialsDto

In [11]:
#json_string = json.dumps([ob.__dict__ for ob in list_name])
#rslt_df = dataframe[dataframe['Percentage'] > 80]
import numpy as np
from pprint import pprint
import os
from dateutil.relativedelta import relativedelta
import json

def npInt_to_str(var):
    return str(list(np.reshape(np.asarray(var), (1, np.size(var)))[0]))[1:-1]

def formatDateNpNum(var):
    dateStr = npInt_to_str(var)
    return dateStr[0:4]+"-"+dateStr[4:6]+"-"+dateStr[6:8]

fileStartTime = datetime.now()
for subId in range(len(dfSub)):
    startTime = datetime.now()
    submitted = dfSub.iloc[subId]
    sfDto = SymbolFinancialsDto()
    sfDto.data = FinancialsDataDto()
    sfDto.data.bs = []
    sfDto.data.cf = []
    sfDto.data.ic = []
    periodStartDate = date.fromisoformat(formatDateNpNum(submitted["period"]))
    sfDto.startDate = periodStartDate
    sfDto.endDate = date.today() #default value
    if submitted["fy"] == np.nan or np.isnan(submitted["fy"]):
        sfDto.year = 0
    else: 
        sfDto.year = submitted["fy"].astype(int)        
    sfDto.quarter = str(submitted["fp"]).strip().upper()
    
    if sfDto.quarter == "FY" or sfDto.quarter == "CY":
        sfDto.endDate = periodStartDate + relativedelta(months=+12, days=-1)
    elif sfDto.quarter == "H1" or sfDto.quarter == "H2":
        sfDto.endDate = periodStartDate + relativedelta(months=+6, days=-1)
    elif sfDto.quarter == "T1" or sfDto.quarter == "T2" or sfDto.quarter == "T3":
        sfDto.endDate = periodStartDate + relativedelta(months=+4, days=-1)
    elif sfDto.quarter == "Q1" or sfDto.quarter == "Q2" or sfDto.quarter == "Q3" or sfDto.quarter == "Q4":
        sfDto.endDate = periodStartDate + relativedelta(months=+3, days=-1)
    else:
        continue
        
    #symStr = dfSym[dfSym["cik"]==submitted.cik].symbol.str.upper().astype("string")
    #print(symStr)
    val = dfSym[dfSym["cik"]==submitted["cik"]]
    if len(val) > 0:
        sfDto.symbol = val["symbol"].to_string(index = False).split("\n")[0].split("\\n")[0].strip().split(" ")[0].strip().upper()
        if len(sfDto.symbol) > 19 or len(sfDto.symbol) < 1:
            print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+sfDto.symbol)
    else:
        print("File exportFiles/"+dirname+"/"+submitted["adsh"]+".json has Symbol "+val["symbol"].to_string(index = False))
        continue
        
    sfDto.name = submitted["name"]
    sfDto.country = submitted["countryma"]
    sfDto.city = submitted["cityma"]

    sdDto = FinancialsDataDto()
    #print(filteredDfPre)
    dfNum['value'] = pd.to_numeric(dfNum['value'], errors='coerce')
    dfNum = dfNum.dropna(subset=['value'])
    dfNum['value'] = dfNum['value'].astype(int)
    #filteredDfNum = dfNum[dfNum['adsh'] == submitted['adsh']]       
    filteredDfNum = dfNum.loc[dfNum['adsh'].values == submitted['adsh']]
    filteredDfNum.reset_index()

    for myId in range(len(filteredDfNum)): 
        #print(myId)
        #print(filteredDfNum.iloc[myId])
        myNum = filteredDfNum.iloc[myId]
        myDto = FinancialElementImportDto()
        myTag = dfTag[dfTag["tag"] == myNum['tag']]
        myDto.label = myTag["doc"].to_string(index = False)
        myDto.concept = myNum["tag"]
        #myPre = dfPre[(dfPre['adsh'] == submitted["adsh"]) & (dfPre['tag'] == myNum['tag'])]
        myPre = dfPre.loc[np.where((dfPre['adsh'].values == submitted["adsh"]) & (dfPre['tag'].values == myNum['tag']))]
        myDto.info = myPre["plabel"].to_string(index = False).replace('"',"'")
        myDto.unit = myNum["uom"]
        #print(myNum["value"])        
        myDto.value = myNum["value"]
        #print(myDto.__dict__)
        FinancialElementImportSchema().dump(myDto)
        if myPre['stmt'].to_string(index = False).strip() == 'BS':
            sfDto.data.bs.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'CF':
            sfDto.data.cf.append(myDto)
        elif myPre['stmt'].to_string(index = False).strip() == 'IC':
            sfDto.data.ic.append(myDto)

    #print(len(filteredDfNum))
    #print(submitted)
    #print(sfDto.__dict__)    
    #json.dumps(sfDto.__dict__)    
    marshmellowStart = datetime.now()
    result = SymbolFinancialsSchema().dump(sfDto)
    result = str(json.dumps(result))
    #result = result.replace('"','öäü').replace("'", '"').replace("öäü","'").replace(", ",",")
    result = result.replace('\\r', '').replace('\\n', ' ')
    
    print("FilterDfNum size: "+str(len(filteredDfNum))+" Dtos size: "+str((len(sfDto.data.bs)+len(sfDto.data.cf)+len(sfDto.data.ic)))+" Bs: "+str(len(sfDto.data.bs))+" Cf: "+str(len(sfDto.data.cf))+" Ic: "+str(len(sfDto.data.ic))+" Json size: "+str(int(len(str(result)) / 1024))+"kb.")
    
    #print("Json size "+str(int(len(str(result)) / 1024))+"kb. Time: "+str((datetime.now() - marshmellowStart) / timedelta(milliseconds=1))+"ms.")
    
    #print(result)
    #pprint(result)    
    isExist = os.path.exists("exportFiles/"+dirname)
    if not isExist:
        os.makedirs("exportFiles/"+dirname)
    json_file = open("exportFiles/"+dirname+"/"+submitted["adsh"]+".json","w")
    json_file.write(result)
    json_file.close()
    endTime = datetime.now()
    print("file "+str(subId+1)+" of "+str(len(dfSub)+1)+" exportFiles/"+dirname+"/"+submitted["adsh"]+".json stored in "+str((endTime - startTime) / timedelta(milliseconds=1))+"ms.")
    
fileEndTime = datetime.now()
print("Time "+str((fileEndTime - fileStartTime) / timedelta(seconds=1))+"sec for "+str(len(dfSub)+1)+" files.")

FilterDfNum size: 177 Dtos size: 88 Bs: 58 Cf: 30 Ic: 0 Json size: 21kb.
file 1 of 6551 exportFiles/2017q4/0000016160-17-000028.json stored in 3011.166ms.
FilterDfNum size: 327 Dtos size: 150 Bs: 74 Cf: 76 Ic: 0 Json size: 35kb.
file 2 of 6551 exportFiles/2017q4/0000045919-17-000011.json stored in 5431.267ms.
File exportFiles/2017q4/0001059272-17-000037.json has Symbol Series([], )
FilterDfNum size: 227 Dtos size: 111 Bs: 49 Cf: 62 Ic: 0 Json size: 26kb.
file 4 of 6551 exportFiles/2017q4/0001144204-17-050946.json stored in 3880.807ms.
FilterDfNum size: 64 Dtos size: 43 Bs: 30 Cf: 13 Ic: 0 Json size: 9kb.
file 5 of 6551 exportFiles/2017q4/0001165527-17-000200.json stored in 1193.617ms.
FilterDfNum size: 74 Dtos size: 41 Bs: 31 Cf: 10 Ic: 0 Json size: 9kb.
file 6 of 6551 exportFiles/2017q4/0001165527-17-000202.json stored in 1270.692ms.
FilterDfNum size: 80 Dtos size: 42 Bs: 29 Cf: 13 Ic: 0 Json size: 9kb.
file 7 of 6551 exportFiles/2017q4/0001165527-17-000204.json stored in 1386.482ms.


FilterDfNum size: 245 Dtos size: 79 Bs: 40 Cf: 39 Ic: 0 Json size: 19kb.
file 69 of 6551 exportFiles/2017q4/0001279569-17-001939.json stored in 4955.504ms.
FilterDfNum size: 166 Dtos size: 81 Bs: 47 Cf: 34 Ic: 0 Json size: 19kb.
file 70 of 6551 exportFiles/2017q4/0001393905-17-000305.json stored in 3591.239ms.
File exportFiles/2017q4/0001477932-17-004889.json has Symbol Series([], )
File exportFiles/2017q4/0001477932-17-004894.json has Symbol Series([], )
File exportFiles/2017q4/0001493152-17-011268.json has Symbol Series([], )
FilterDfNum size: 159 Dtos size: 51 Bs: 29 Cf: 22 Ic: 0 Json size: 12kb.
file 75 of 6551 exportFiles/2017q4/0001493152-17-011311.json stored in 3628.905ms.
File exportFiles/2017q4/0001625376-17-000024.json has Symbol Series([], )
FilterDfNum size: 206 Dtos size: 104 Bs: 48 Cf: 56 Ic: 0 Json size: 24kb.
file 79 of 6551 exportFiles/2017q4/0000320187-17-000184.json stored in 4729.885ms.
FilterDfNum size: 278 Dtos size: 139 Bs: 79 Cf: 60 Ic: 0 Json size: 32kb.
file 

FilterDfNum size: 147 Dtos size: 96 Bs: 63 Cf: 33 Ic: 0 Json size: 22kb.
file 141 of 6551 exportFiles/2017q4/0001213900-17-010490.json stored in 4386.075ms.
FilterDfNum size: 188 Dtos size: 81 Bs: 47 Cf: 34 Ic: 0 Json size: 19kb.
file 142 of 6551 exportFiles/2017q4/0001477932-17-004971.json stored in 4404.735ms.
File exportFiles/2017q4/0001640334-17-002152.json has Symbol Series([], )
FilterDfNum size: 382 Dtos size: 139 Bs: 75 Cf: 64 Ic: 0 Json size: 32kb.
file 145 of 6551 exportFiles/2017q4/0000006845-17-000021.json stored in 8895.763ms.
FilterDfNum size: 115 Dtos size: 76 Bs: 44 Cf: 32 Ic: 0 Json size: 17kb.
file 146 of 6551 exportFiles/2017q4/0000096536-17-000037.json stored in 2856.411ms.
FilterDfNum size: 542 Dtos size: 132 Bs: 60 Cf: 72 Ic: 0 Json size: 30kb.
file 147 of 6551 exportFiles/2017q4/0000103595-17-000020.json stored in 16203.184ms.
FilterDfNum size: 373 Dtos size: 126 Bs: 64 Cf: 62 Ic: 0 Json size: 29kb.
file 148 of 6551 exportFiles/2017q4/0000783005-17-000056.json st

FilterDfNum size: 259 Dtos size: 92 Bs: 66 Cf: 26 Ic: 0 Json size: 21kb.
file 204 of 6551 exportFiles/2017q4/0001493152-17-011592.json stored in 10938.731ms.
FilterDfNum size: 174 Dtos size: 69 Bs: 38 Cf: 31 Ic: 0 Json size: 15kb.
file 205 of 6551 exportFiles/2017q4/0001520138-17-000323.json stored in 7993.313ms.
File exportFiles/2017q4/0001552781-17-000556.json has Symbol Series([], )
File exportFiles/2017q4/0001564590-17-019472.json has Symbol Series([], )
FilterDfNum size: 602 Dtos size: 158 Bs: 57 Cf: 101 Ic: 0 Json size: 38kb.
file 208 of 6551 exportFiles/2017q4/0001564590-17-019480.json stored in 26970.67ms.
File exportFiles/2017q4/0001594062-17-000234.json has Symbol Series([], )
File exportFiles/2017q4/0001615774-17-005720.json has Symbol Series([], )
FilterDfNum size: 140 Dtos size: 72 Bs: 44 Cf: 28 Ic: 0 Json size: 17kb.
file 211 of 6551 exportFiles/2017q4/0001615774-17-005727.json stored in 6539.42ms.
FilterDfNum size: 55 Dtos size: 35 Bs: 25 Cf: 10 Ic: 0 Json size: 8kb.
fil

FilterDfNum size: 95 Dtos size: 64 Bs: 50 Cf: 14 Ic: 0 Json size: 15kb.
file 271 of 6551 exportFiles/2017q4/0001654954-17-009449.json stored in 4577.025ms.
File exportFiles/2017q4/0001654954-17-009481.json has Symbol Series([], )
FilterDfNum size: 208 Dtos size: 106 Bs: 62 Cf: 44 Ic: 0 Json size: 25kb.
file 273 of 6551 exportFiles/2017q4/0001654954-17-009482.json stored in 9677.996ms.
File exportFiles/2017q4/0001659173-17-000456.json has Symbol Series([], )
File exportFiles/2017q4/0001662252-17-000212.json has Symbol Series([], )
FilterDfNum size: 72 Dtos size: 42 Bs: 24 Cf: 18 Ic: 0 Json size: 10kb.
file 276 of 6551 exportFiles/2017q4/0001678848-17-000018.json stored in 3658.427ms.
File exportFiles/2017q4/0001683168-17-002659.json has Symbol Series([], )
FilterDfNum size: 131 Dtos size: 80 Bs: 56 Cf: 24 Ic: 0 Json size: 18kb.
file 278 of 6551 exportFiles/2017q4/0001683168-17-002665.json stored in 6337.232ms.
File exportFiles/2017q4/0001683168-17-002671.json has Symbol Series([], )
Fil

...

FilterDfNum size: 164 Dtos size: 73 Bs: 44 Cf: 29 Ic: 0 Json size: 16kb.
file 6277 of 6551 exportFiles/2017q4/0001393905-17-000399.json stored in 5807.267ms.
FilterDfNum size: 131 Dtos size: 78 Bs: 46 Cf: 32 Ic: 0 Json size: 18kb.
file 6278 of 6551 exportFiles/2017q4/0001393905-17-000402.json stored in 4157.308ms.
File exportFiles/2017q4/0001424844-17-000015.json has Symbol Series([], )
FilterDfNum size: 199 Dtos size: 92 Bs: 59 Cf: 33 Ic: 0 Json size: 20kb.
file 6280 of 6551 exportFiles/2017q4/0001437749-17-020734.json stored in 5378.529ms.
FilterDfNum size: 258 Dtos size: 133 Bs: 94 Cf: 39 Ic: 0 Json size: 30kb.
file 6281 of 6551 exportFiles/2017q4/0001437749-17-020750.json stored in 8415.829ms.
FilterDfNum size: 627 Dtos size: 171 Bs: 78 Cf: 93 Ic: 0 Json size: 41kb.
file 6282 of 6551 exportFiles/2017q4/0001437749-17-020777.json stored in 18892.367ms.
File exportFiles/2017q4/0001437749-17-020789.json has Symbol Series([], )
FilterDfNum size: 294 Dtos size: 114 Bs: 58 Cf: 56 Ic: 0 Js

FilterDfNum size: 451 Dtos size: 133 Bs: 73 Cf: 60 Ic: 0 Json size: 31kb.
file 6342 of 6551 exportFiles/2017q4/0001144204-17-064208.json stored in 13036.625ms.
FilterDfNum size: 233 Dtos size: 113 Bs: 66 Cf: 47 Ic: 0 Json size: 26kb.
file 6343 of 6551 exportFiles/2017q4/0001193125-17-373692.json stored in 6440.55ms.
FilterDfNum size: 130 Dtos size: 69 Bs: 47 Cf: 22 Ic: 0 Json size: 16kb.
file 6344 of 6551 exportFiles/2017q4/0001213900-17-013389.json stored in 3739.762ms.
File exportFiles/2017q4/0001477932-17-006166.json has Symbol Series([], )
FilterDfNum size: 160 Dtos size: 89 Bs: 62 Cf: 27 Ic: 0 Json size: 21kb.
file 6347 of 6551 exportFiles/2017q4/0001493152-17-014749.json stored in 4470.293ms.
FilterDfNum size: 177 Dtos size: 75 Bs: 51 Cf: 24 Ic: 0 Json size: 17kb.
file 6348 of 6551 exportFiles/2017q4/0001511164-17-000763.json stored in 5504.856ms.
FilterDfNum size: 258 Dtos size: 134 Bs: 71 Cf: 63 Ic: 0 Json size: 31kb.
file 6349 of 6551 exportFiles/2017q4/0001558370-17-009392.js

FilterDfNum size: 247 Dtos size: 138 Bs: 84 Cf: 54 Ic: 0 Json size: 32kb.
file 6410 of 6551 exportFiles/2017q4/0000909832-17-000022.json stored in 6746.645ms.
FilterDfNum size: 137 Dtos size: 88 Bs: 52 Cf: 36 Ic: 0 Json size: 20kb.
file 6411 of 6551 exportFiles/2017q4/0001078782-17-001713.json stored in 4373.063ms.
File exportFiles/2017q4/0001079974-17-000574.json has Symbol Series([], )
FilterDfNum size: 869 Dtos size: 189 Bs: 72 Cf: 117 Ic: 0 Json size: 43kb.
file 6413 of 6551 exportFiles/2017q4/0001090872-17-000018.json stored in 35894.069ms.
File exportFiles/2017q4/0001157523-17-003265.json has Symbol Series([], )
FilterDfNum size: 603 Dtos size: 144 Bs: 68 Cf: 76 Ic: 0 Json size: 33kb.
file 6417 of 6551 exportFiles/2017q4/0001174947-17-001813.json stored in 29435.331ms.
FilterDfNum size: 374 Dtos size: 134 Bs: 69 Cf: 65 Ic: 0 Json size: 30kb.
file 6418 of 6551 exportFiles/2017q4/0001174947-17-001818.json stored in 18424.118ms.
FilterDfNum size: 251 Dtos size: 117 Bs: 69 Cf: 48 Ic:

FilterDfNum size: 69 Dtos size: 23 Bs: 19 Cf: 4 Ic: 0 Json size: 5kb.
file 6476 of 6551 exportFiles/2017q4/0001086303-17-000024.json stored in 2229.69ms.
FilterDfNum size: 78 Dtos size: 28 Bs: 22 Cf: 6 Ic: 0 Json size: 6kb.
file 6477 of 6551 exportFiles/2017q4/0001086303-17-000029.json stored in 2246.962ms.
FilterDfNum size: 78 Dtos size: 40 Bs: 27 Cf: 13 Ic: 0 Json size: 9kb.
file 6478 of 6551 exportFiles/2017q4/0001165527-17-000246.json stored in 2286.106ms.
FilterDfNum size: 534 Dtos size: 19 Bs: 4 Cf: 15 Ic: 0 Json size: 4kb.
file 6479 of 6551 exportFiles/2017q4/0001398432-17-000185.json stored in 15387.848ms.
File exportFiles/2017q4/0001493152-17-015041.json has Symbol Series([], )
File exportFiles/2017q4/0001564590-17-025311.json has Symbol Series([], )
FilterDfNum size: 357 Dtos size: 188 Bs: 84 Cf: 104 Ic: 0 Json size: 39kb.
file 6482 of 6551 exportFiles/2017q4/0001654954-17-011858.json stored in 9725.909ms.
FilterDfNum size: 288 Dtos size: 131 Bs: 75 Cf: 56 Ic: 0 Json size: 30

FilterDfNum size: 583 Dtos size: 151 Bs: 76 Cf: 75 Ic: 0 Json size: 35kb.
file 6543 of 6551 exportFiles/2017q4/0001615774-17-007705.json stored in 15419.773ms.
FilterDfNum size: 326 Dtos size: 132 Bs: 68 Cf: 64 Ic: 0 Json size: 29kb.
file 6544 of 6551 exportFiles/2017q4/0001654954-17-011928.json stored in 8888.976ms.
FilterDfNum size: 258 Dtos size: 138 Bs: 62 Cf: 76 Ic: 0 Json size: 30kb.
file 6545 of 6551 exportFiles/2017q4/0001654954-17-011929.json stored in 7952.222ms.
File exportFiles/2017q4/0001662252-17-000254.json has Symbol Series([], )
File exportFiles/2017q4/0001683168-17-003424.json has Symbol Series([], )
FilterDfNum size: 160 Dtos size: 68 Bs: 43 Cf: 25 Ic: 0 Json size: 16kb.
file 6549 of 6551 exportFiles/2017q4/0001721868-17-000186.json stored in 5096.203ms.
FilterDfNum size: 167 Dtos size: 69 Bs: 43 Cf: 26 Ic: 0 Json size: 16kb.
file 6550 of 6551 exportFiles/2017q4/0001721868-17-000188.json stored in 5985.915ms.
Time 52621.675773sec for 6551 files.
